# Worksheet 7 - Classification (Part II)

### Lecture and Tutorial Learning Goals:

After completing this week's lecture and tutorial work, you will be able to:

* Describe what a test data set is and how it is used in classification.
* Using Python, evaluate classification accuracy using a test data set and appropriate metrics.
* Using Python, execute cross-validation in Python to choose the number of neighbours.
* Identify when it is necessary to scale variables before classification and do this using Python
* In a dataset with > 2 attributes, perform k-nearest neighbour classification in Python using the `scikit-learn` package to predict the class of a test dataset.
* Describe advantages and disadvantages of the k-nearest neighbour classification algorithm.

This worksheet covers parts of [Chapter 6](https://python.datasciencebook.ca/classification2) of the online textbook. You should read this chapter before attempting this assignment. Any place you see `___`, you must fill in the function, variable, or data to complete the code. Substitute the `raise NotImplementedError` with your completed code and answers then proceed to run the cell!

In [ ]:
### Run this cell before continuing.
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

**Question 0.1** Multiple Choice:
<br>{points: 1}

Before applying k-nearest neighbour to a classification task, we need to scale the data. What is the purpose of this step?

A. To help speed up the knn algorithm.

B. To convert all data observations to numeric values.

C. To ensure all data observations will be on a comparable scale and contribute equal shares to the calculation of the distance between points.

D. None of the above.

*Assign your answer to an object called `answer0_1`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`)*.

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer0_1)).encode("utf-8")+b"59e8").hexdigest() == "f70a6af77babf11afa3a50f7a15e532aa7baf3f3", "type of answer0_1 is not str. answer0_1 should be an str"
assert sha1(str(len(answer0_1)).encode("utf-8")+b"59e8").hexdigest() == "3b3b8f9d183db109adfa13225d741063b08dff85", "length of answer0_1 is not correct"
assert sha1(str(answer0_1.lower()).encode("utf-8")+b"59e8").hexdigest() == "6edfdac10655fefa98b28f34c9d1f860dcce8ff7", "value of answer0_1 is not correct"
assert sha1(str(answer0_1).encode("utf-8")+b"59e8").hexdigest() == "9dd39830e122b8184b1799ac7d2e687f95a317b7", "correct string value of answer0_1 but incorrect case of letters"

print('Success!')

## 1. Fruit Data Example - (Part II)
**Question 1.0** 
<br>{points: 1}

You will recognize a few of the first questions in this worksheet from last week's tutorial. This will help you repeat some of the fundamentals of classification before tackling the later questions in this worksheet, which integrate concepts you learned from this week's material. First, load the file `fruit_data.csv` (found in the data folder) from the previous tutorial into your notebook.

*Assign your data to an object called `fruit_data`.*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_data is None)).encode("utf-8")+b"d063d").hexdigest() == "a14058557363854dce700043f5f8c1bdf382a646", "type of fruit_data is None is not bool. fruit_data is None should be a bool"
assert sha1(str(fruit_data is None).encode("utf-8")+b"d063d").hexdigest() == "6f14f2c9ca926b7709268f219df6f138d1231625", "boolean value of fruit_data is None is not correct"

assert sha1(str(type(fruit_data.shape)).encode("utf-8")+b"d063e").hexdigest() == "f7906d6719d9a050d4d1f3a7c6c618fb002f0fa1", "type of fruit_data.shape is not tuple. fruit_data.shape should be a tuple"
assert sha1(str(len(fruit_data.shape)).encode("utf-8")+b"d063e").hexdigest() == "4ff20a6fbec99bebf47dac2bf82cd01469072d89", "length of fruit_data.shape is not correct"
assert sha1(str(sorted(map(str, fruit_data.shape))).encode("utf-8")+b"d063e").hexdigest() == "e18e39bda9a33957a5949bbb3ce324a011434451", "values of fruit_data.shape are not correct"
assert sha1(str(fruit_data.shape).encode("utf-8")+b"d063e").hexdigest() == "16cb0aff3f219d1a86d4dfe63331a137736d4345", "order of elements of fruit_data.shape is not correct"

assert sha1(str(type(fruit_data.fruit_name.dtype)).encode("utf-8")+b"d063f").hexdigest() == "ecb25c014845e8f6ff8c01f6d46cad4bc1c87e09", "type of fruit_data.fruit_name.dtype is not correct"
assert sha1(str(fruit_data.fruit_name.dtype).encode("utf-8")+b"d063f").hexdigest() == "297d15d689bbf3e05f25a68b218276c17d16cd00", "value of fruit_data.fruit_name.dtype is not correct"

print('Success!')

Let's take a look at the first few observations in the fruit dataset. Run the cell below.

In [ ]:
# Run this cell.
fruit_data.head()

Run the cell below, and find the nearest neighbour based on mass and width to the first observation just by looking at the scatterplot (the first observation has been circled for you).

In [ ]:
# Create the scatterplot
fruit_chart = alt.Chart(fruit_data).mark_point(size=15).encode(
    x=alt.X("mass").title("Mass (grams)"),
    y=alt.Y("width")
        .title("Width (cm)")
        .scale(zero=False),
    color=alt.Color("fruit_name").title("Fruit")
)
# Create the circle around point1
point1 = alt.Chart().mark_point(size=300, color='black').encode(
    x=alt.datum(fruit_data.loc[0, "mass"]),
    y=alt.datum(fruit_data.loc[0, "width"]),
    text=alt.datum('1')
)

# Layer the scatterplot, point1 circle, and a text annotation
fruit_chart + point1 + point1.mark_text(size=14, dx=12, align='left')

**Question 1.1** Multiple Choice:
<br>{points: 1}

Based on the graph generated, what is the `fruit_name` of the closest data point to the one circled?

A. apple

B. lemon

C. mandarin

D. orange

*Assign your answer to an object called `answer1_1`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_1)).encode("utf-8")+b"70796").hexdigest() == "6ac80748d62e764539df40ada7f54547f9189871", "type of answer1_1 is not str. answer1_1 should be an str"
assert sha1(str(len(answer1_1)).encode("utf-8")+b"70796").hexdigest() == "28ae52bc882eefa1093d8c42163f449c588ee942", "length of answer1_1 is not correct"
assert sha1(str(answer1_1.lower()).encode("utf-8")+b"70796").hexdigest() == "10f1d8ad7a7af04228b4e34c5678b943271022d9", "value of answer1_1 is not correct"
assert sha1(str(answer1_1).encode("utf-8")+b"70796").hexdigest() == "b5a1a1a5392aecd9062b4d2915fa623f38d18dc6", "correct string value of answer1_1 but incorrect case of letters"

print('Success!')

**Question 1.2**
<br>{points: 1}

Using `mass` and `width`, calculate the distance between the first observation and the second observation.

We provide a scaffolding to get you started.

*Assign your answer to an object called `fruit_dist_2`.*

In [ ]:
# ___ = euclidean_distances(fruit_data.loc[[___, 1], ["mass", ___]])

# your code here
raise NotImplementedError
fruit_dist_2

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_dist_2)).encode("utf-8")+b"e519d").hexdigest() == "26bc3b3326d2c5b8600ba2b3dc0df77fb895402e", "type of fruit_dist_2 is not correct"
assert sha1(str(fruit_dist_2).encode("utf-8")+b"e519d").hexdigest() == "deb078fbd954577349f09ed06c58a413d591ddf0", "value of fruit_dist_2 is not correct"

print('Success!')

**Question 1.3**
<br>{points: 1}

Calculate the distance between the first and the **44th observation** in the `fruit` dataset using the `mass` and `width` variables.

*Assign your answer to an object called `fruit_dist_44`.*

In [ ]:
# your code here
raise NotImplementedError
fruit_dist_44

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_dist_44)).encode("utf-8")+b"eeb4d").hexdigest() == "739d464296eecbea7b573d6bd46f717299c93caf", "type of fruit_dist_44 is not correct"
assert sha1(str(fruit_dist_44).encode("utf-8")+b"eeb4d").hexdigest() == "5be4fec3266af42e1f4f833637b06f255ad2669b", "value of fruit_dist_44 is not correct"

print('Success!')

What do you notice about your answers from **Question 1.2** & **1.3** that you just calculated? Is it what you would expect given the scatter plot below? Why or why not? Discuss this.

*Hint: Look at where the observations are on the scatterplot in the cell below this question. What might happen if we changed grams into kilograms to measure the mass?*

In [ ]:
# Run this cell to see the scatter plot
points = alt.Chart(
    fruit_data.loc[[0, 1, 43], ['mass', 'width']].assign(point_number=[1, 2, 44])
).mark_point(size=300, color='black').encode(
    x='mass',
    y='width',
    text='point_number'
)

fruit_chart + points + points.mark_text(size=14, dx=12, align='left')

**Question 1.4**
<br>{points: 1}

From the distance calculation, we see that observation 1 and 44 have a smaller distance than observation 1 and 2. However, if we look at the scatterplot the distance of the first observation to the second observation appears closer than to the 44th observation.

Which of the following statements is correct?

A. A difference of 12 g in mass between observation 1 and 2 is large compared to a difference of 1.2 cm in width between observation 1 and 44. Consequently, mass will drive the classification results, and width will have less of an effect. Hence, our distance calculation reflects that.

B. If we measured mass in kilograms, then we’d get different classification results.

C. We should standardize the data so that all variables will be on a comparable scale.

D. All of the above.

*Assign your answer to an object called `answer1_4`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_4)).encode("utf-8")+b"21e4f").hexdigest() == "5a9939d3c8289c5a6fef60e643fcdda9f9eb5f43", "type of answer1_4 is not str. answer1_4 should be an str"
assert sha1(str(len(answer1_4)).encode("utf-8")+b"21e4f").hexdigest() == "1d9a335db65e0f978544df3ef138a6922c1f4dfc", "length of answer1_4 is not correct"
assert sha1(str(answer1_4.lower()).encode("utf-8")+b"21e4f").hexdigest() == "d38e23ea2528aab392e4186de0508d42f2d4344d", "value of answer1_4 is not correct"
assert sha1(str(answer1_4).encode("utf-8")+b"21e4f").hexdigest() == "22f33f3b8c2814c92be2673d19cf746f0de5876c", "correct string value of answer1_4 but incorrect case of letters"

print('Success!')

**Question 1.5**
<br> {points: 1}

Let's create a `preprocessor` to *standardize* (i.e., center and scale) all of the variables in the fruit dataset. Centering will make sure that every variable has an average of 0, and scaling will make sure that every variable has standard deviation of 1. We will use the `StandardScaler` in the `preprocessor`. Then `fit_transform` the preprocessor so that we can examine the output.

Fit and transform your preprocessor with predictors `mass`, `width`, `height`, and `color_score`. Pass through all the remaining columns.

Name the preprocessor `fruit_data_preprocessor`, and name the preprocessed data frame `fruit_data_scaled`.

In [ ]:
# ___ = ___(
#     (StandardScaler(), [___, ___, ___, ___]),
#     remainder=___
# )

# ___ = fruit_data_preprocessor.___(___)


# your code here
raise NotImplementedError
fruit_data_scaled.head()

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_data_scaled is None)).encode("utf-8")+b"3c8cb").hexdigest() == "ff9a08d4b20a786c3fe0c629a06e417c949a14b6", "type of fruit_data_scaled is None is not bool. fruit_data_scaled is None should be a bool"
assert sha1(str(fruit_data_scaled is None).encode("utf-8")+b"3c8cb").hexdigest() == "f0f516bb8a32d85e11338beb0529030f8eac49d1", "boolean value of fruit_data_scaled is None is not correct"

assert sha1(str(type(fruit_data_scaled.shape)).encode("utf-8")+b"3c8cc").hexdigest() == "df0c5ff699ea164e1863ba6229927f07d26bb47f", "type of fruit_data_scaled.shape is not tuple. fruit_data_scaled.shape should be a tuple"
assert sha1(str(len(fruit_data_scaled.shape)).encode("utf-8")+b"3c8cc").hexdigest() == "72007e085f834c05f4953538bec73ae80a5cbce9", "length of fruit_data_scaled.shape is not correct"
assert sha1(str(sorted(map(str, fruit_data_scaled.shape))).encode("utf-8")+b"3c8cc").hexdigest() == "7ccc4a17db1fc608788b4a5e9847c3ae9742ecdb", "values of fruit_data_scaled.shape are not correct"
assert sha1(str(fruit_data_scaled.shape).encode("utf-8")+b"3c8cc").hexdigest() == "a1ef72e7c1ce0d9e5ecb18f37e3692f94b657138", "order of elements of fruit_data_scaled.shape is not correct"

assert sha1(str(type(fruit_data_scaled.fruit_name.dtype)).encode("utf-8")+b"3c8cd").hexdigest() == "1364e000ac4a39a6266a39f3344654a24e2ebeff", "type of fruit_data_scaled.fruit_name.dtype is not correct"
assert sha1(str(fruit_data_scaled.fruit_name.dtype).encode("utf-8")+b"3c8cd").hexdigest() == "f02aef11f881c1d13e7db1891779840ae1dca2fc", "value of fruit_data_scaled.fruit_name.dtype is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.mass.dropna()))).encode("utf-8")+b"3c8ce").hexdigest() == "5af92ae04bdccb61418a85b68d13bd206c80a864", "type of np.mean(fruit_data_scaled.mass.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.mass.dropna())).encode("utf-8")+b"3c8ce").hexdigest() == "57022ebf91a808ded6cbf0d622abc75ddb14676d", "value of np.mean(fruit_data_scaled.mass.dropna()) is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.height.dropna()))).encode("utf-8")+b"3c8cf").hexdigest() == "bcc05b239f78f4f04767e21203a256845ee11cf6", "type of np.mean(fruit_data_scaled.height.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.height.dropna())).encode("utf-8")+b"3c8cf").hexdigest() == "eeef10013313e114d8f45c0565759a44c232783f", "value of np.mean(fruit_data_scaled.height.dropna()) is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.width.dropna()))).encode("utf-8")+b"3c8d0").hexdigest() == "f1d39b610b4cb9f4e1cc6af9b67076cdc7787577", "type of np.mean(fruit_data_scaled.width.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.width.dropna())).encode("utf-8")+b"3c8d0").hexdigest() == "194c3e038f375a1d80190ceba3eb9162a54a49dd", "value of np.mean(fruit_data_scaled.width.dropna()) is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.color_score.dropna()))).encode("utf-8")+b"3c8d1").hexdigest() == "fa17251dfa91bf285b37cc7f701c0c628a31b082", "type of np.mean(fruit_data_scaled.color_score.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.color_score.dropna())).encode("utf-8")+b"3c8d1").hexdigest() == "be1ff7b84f82426150b5f0c453e5b29b4798b04e", "value of np.mean(fruit_data_scaled.color_score.dropna()) is not correct"

assert sha1(str(type(np.std(fruit_data_scaled.mass.dropna()))).encode("utf-8")+b"3c8d2").hexdigest() == "ab19602ab38a54621d9a4c4ac2a5c89603ec4a47", "type of np.std(fruit_data_scaled.mass.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.mass.dropna()), 2)).encode("utf-8")+b"3c8d2").hexdigest() == "cd935a9c9f3ede182b96f7b8d8e75cbf654f3880", "value of np.std(fruit_data_scaled.mass.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(np.std(fruit_data_scaled.height.dropna()))).encode("utf-8")+b"3c8d3").hexdigest() == "0bdc778880bd321d0b1eb20c711b952485a439b1", "type of np.std(fruit_data_scaled.height.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.height.dropna()), 2)).encode("utf-8")+b"3c8d3").hexdigest() == "464894f3d5f5b9e5eb560e3245a4290960c245a3", "value of np.std(fruit_data_scaled.height.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(np.std(fruit_data_scaled.width.dropna()))).encode("utf-8")+b"3c8d4").hexdigest() == "9c4e165a8f7b228d0f8249940429df13b690f676", "type of np.std(fruit_data_scaled.width.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.width.dropna()), 2)).encode("utf-8")+b"3c8d4").hexdigest() == "d0a1e7ddfd49bb9908f2f9c7b19512535c2f5fa7", "value of np.std(fruit_data_scaled.width.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(np.std(fruit_data_scaled.color_score.dropna()))).encode("utf-8")+b"3c8d5").hexdigest() == "c60c23026557d8372c3554cdc79513a6559bae02", "type of np.std(fruit_data_scaled.color_score.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.color_score.dropna()), 2)).encode("utf-8")+b"3c8d5").hexdigest() == "3fbb866caf987e4d08c0823885b16f0b5273231e", "value of np.std(fruit_data_scaled.color_score.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(fruit_data_preprocessor is None)).encode("utf-8")+b"3c8d6").hexdigest() == "bbd557c1b8b813fe77d99d1a281201b0ed293ce8", "type of fruit_data_preprocessor is None is not bool. fruit_data_preprocessor is None should be a bool"
assert sha1(str(fruit_data_preprocessor is None).encode("utf-8")+b"3c8d6").hexdigest() == "58b0515dd1178f19494b0605d36f43089cad24e8", "boolean value of fruit_data_preprocessor is None is not correct"

assert sha1(str(type(fruit_data_preprocessor.transformers_[1][2])).encode("utf-8")+b"3c8d7").hexdigest() == "23a02732b405ad3ca734164996325400213c5ba1", "type of fruit_data_preprocessor.transformers_[1][2] is not list. fruit_data_preprocessor.transformers_[1][2] should be a list"
assert sha1(str(len(fruit_data_preprocessor.transformers_[1][2])).encode("utf-8")+b"3c8d7").hexdigest() == "f6865c38936e8038b24291bc3fcfeb51620d3248", "length of fruit_data_preprocessor.transformers_[1][2] is not correct"
assert sha1(str(sorted(map(str, fruit_data_preprocessor.transformers_[1][2]))).encode("utf-8")+b"3c8d7").hexdigest() == "f39f713962245fee7e0f7cb2a7d034bab72414e9", "values of fruit_data_preprocessor.transformers_[1][2] are not correct"
assert sha1(str(fruit_data_preprocessor.transformers_[1][2]).encode("utf-8")+b"3c8d7").hexdigest() == "117f8891bedffdeb17eba1c9db2feb0a490c16d6", "order of elements of fruit_data_preprocessor.transformers_[1][2] is not correct"

print('Success!')

**Question 1.6**
<br> {points: 1}

Let's repeat **Question 1.2 and 1.3** with the scaled variables:

- calculate the distance with the scaled mass and width variables between observations 1 and 2
- calculate the distances with the scaled mass and width variables between observations 1 and 44 

After you do this, think about how these distances compared to the distances you computed in **Question 1.2 and 1.3** for the same points.

*Assign your answers to objects called `distance_2` and `distance_44` respectively.*

In [ ]:
# your code here
raise NotImplementedError
print(distance_2)
print(distance_44)

In [ ]:
from hashlib import sha1
assert sha1(str(type(distance_2 is None)).encode("utf-8")+b"ec7a8").hexdigest() == "c346c808e5ab3b40222c82112df523ecf9c22c76", "type of distance_2 is None is not bool. distance_2 is None should be a bool"
assert sha1(str(distance_2 is None).encode("utf-8")+b"ec7a8").hexdigest() == "1404d9b5db6d0ef28e0d547ea76a791252c88d1d", "boolean value of distance_2 is None is not correct"

assert sha1(str(type(distance_44 is None)).encode("utf-8")+b"ec7a9").hexdigest() == "06f14e8dcaeed5f4ac8b6f12961e7cefd02116c0", "type of distance_44 is None is not bool. distance_44 is None should be a bool"
assert sha1(str(distance_44 is None).encode("utf-8")+b"ec7a9").hexdigest() == "83228d05c85f9ea2a837a2f19c08d6646c7ead79", "boolean value of distance_44 is None is not correct"

assert sha1(str(type(distance_2)).encode("utf-8")+b"ec7aa").hexdigest() == "e0480c79eb5aace41e9523012e86611102f6d327", "type of type(distance_2) is not correct"

assert sha1(str(type(distance_44)).encode("utf-8")+b"ec7ab").hexdigest() == "bb4835f4bc9085771b3e7b74621427131a72ba6b", "type of type(distance_44) is not correct"

assert sha1(str(type(distance_2)).encode("utf-8")+b"ec7ac").hexdigest() == "782845f48c8394e602be987fd7c81c759811d31b", "type of distance_2 is not correct"
assert sha1(str(distance_2).encode("utf-8")+b"ec7ac").hexdigest() == "30a769b810cf07187bee2b7721438ee53eabf0f4", "value of distance_2 is not correct"

assert sha1(str(type(distance_44)).encode("utf-8")+b"ec7ad").hexdigest() == "97d48bde2037795499f0aaa763e6abc264c7ed46", "type of distance_44 is not correct"
assert sha1(str(distance_44).encode("utf-8")+b"ec7ad").hexdigest() == "d7b5245ce6d43585679782e568dd03958bb6e4ac", "value of distance_44 is not correct"

print('Success!')

## Randomness and Setting Seeds

This worksheet uses functions from the `scikit-learn` library, which not only allows us to perform K-nearest neighbour classification, but also allows us to evaluate how well our classification worked. In order to ensure that the steps in the worksheet are reproducible, we need to set a *`random_state`* or *random seed*, i.e., a numerical "starting value," which determines the sequence of random numbers Python will generate.

Below in many cells we have included an argument to set the `random_state` or `np.random.seed`. They are necessary to make sure the autotesting code functions properly.

## 2. Splitting the data into a training and test set

In this exercise, we will be partitioning `fruit_data` into a training (75%) and testing (25%) set using the `scikit-learn` package. After creating the test set, we will put the test set away in a lock box and not touch it again until we have found the best k-nn classifier we can make using the training set. We will use the variable `fruit_name` as our class label. 


**Question 2.0**
<br> {points: 1}

To create the training and test set, we would use the `train_test_split` function from `scikit-learn` pacakge. Save the trained dataset and test dataset as `fruit_train` and `fruit_test`, respectively. 

In [ ]:
# Randomly take 75% of the data in the training set.
# This will be proportional to the different number of fruit names in the dataset.

# ___, ___ = train_test_split(___, test_size=___, random_state=123) # set the random state to be 123

# your code here
raise NotImplementedError
fruit_train

In [ ]:
fruit_test

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_train is None)).encode("utf-8")+b"6c3bf").hexdigest() == "396c981f060f2edf7d6d66b208d437b8196a39b3", "type of fruit_train is None is not bool. fruit_train is None should be a bool"
assert sha1(str(fruit_train is None).encode("utf-8")+b"6c3bf").hexdigest() == "ec46abacf6dc2b55fc7b7ab55a1ff541b8a4ce98", "boolean value of fruit_train is None is not correct"

assert sha1(str(type(fruit_test is None)).encode("utf-8")+b"6c3c0").hexdigest() == "a561ecaac2aa8a361d496f1f4c753852f6a998d6", "type of fruit_test is None is not bool. fruit_test is None should be a bool"
assert sha1(str(fruit_test is None).encode("utf-8")+b"6c3c0").hexdigest() == "a90c63d59f1f9f5ff5ae19d8c5af4648c0635dec", "boolean value of fruit_test is None is not correct"

assert sha1(str(type(fruit_train.shape)).encode("utf-8")+b"6c3c1").hexdigest() == "5ac7082745d4a5049e9da91595e9ff1dd42692c8", "type of fruit_train.shape is not tuple. fruit_train.shape should be a tuple"
assert sha1(str(len(fruit_train.shape)).encode("utf-8")+b"6c3c1").hexdigest() == "777a55e8236a916b4595aec545e95d66f6354f2a", "length of fruit_train.shape is not correct"
assert sha1(str(sorted(map(str, fruit_train.shape))).encode("utf-8")+b"6c3c1").hexdigest() == "9d26d021fbc23c59a5677c09cba052af1f81f25b", "values of fruit_train.shape are not correct"
assert sha1(str(fruit_train.shape).encode("utf-8")+b"6c3c1").hexdigest() == "a889acc95ca863c25ae987b94ae56afaa54b767d", "order of elements of fruit_train.shape is not correct"

assert sha1(str(type(fruit_test.shape)).encode("utf-8")+b"6c3c2").hexdigest() == "cba744002cd1af057a9014161046f153d8fd02ab", "type of fruit_test.shape is not tuple. fruit_test.shape should be a tuple"
assert sha1(str(len(fruit_test.shape)).encode("utf-8")+b"6c3c2").hexdigest() == "9d5a8b3727cf5d1e7cd838d6c45d7db79b0dce61", "length of fruit_test.shape is not correct"
assert sha1(str(sorted(map(str, fruit_test.shape))).encode("utf-8")+b"6c3c2").hexdigest() == "d9c9ee73395afd2f186b8bba8afc7c3366c660e0", "values of fruit_test.shape are not correct"
assert sha1(str(fruit_test.shape).encode("utf-8")+b"6c3c2").hexdigest() == "a7d89c7de4f7b0f524014bfcc18f7caf822b6ba6", "order of elements of fruit_test.shape is not correct"

assert sha1(str(type(sum(fruit_train.mass))).encode("utf-8")+b"6c3c3").hexdigest() == "2b8ae85e8040d6d2818b9f979546e6689c438bf5", "type of sum(fruit_train.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(fruit_train.mass)).encode("utf-8")+b"6c3c3").hexdigest() == "75c62028dd3ff2dcd9dcab547a7a59d97d2a73bd", "value of sum(fruit_train.mass) is not correct"

assert sha1(str(type(sum(fruit_test.mass))).encode("utf-8")+b"6c3c4").hexdigest() == "8697751b07db9c24037f7ad9cccdf440bb8e42dc", "type of sum(fruit_test.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(fruit_test.mass)).encode("utf-8")+b"6c3c4").hexdigest() == "9fb8d06ac20cd09354a876c611c834522f0171dd", "value of sum(fruit_test.mass) is not correct"

print('Success!')

**Question 2.1** 
<br> {points: 1}

K-nearest neighbors is sensitive to the scale of the predictors so we should do some preprocessing to standardize them. Remember that standardizing involves centering/shifting (subtracting the mean of each variable) and scaling (dividing by its standard deviation). Also remember that standardization is *part of your training procedure*, so you can't use your test data to compute the centered / scaled values for each variable. Therefore, you must pass only the training data to your preprocessor to compute the preprocessing steps. This ensures that our test data does not influence any aspect of our model training. Once we have created the standardization preprocessor, we can then later on apply it separately to both the training and test data sets.

For this exercise, let's see if `mass` and `color_score` can predict `fruit_name`. 

To scale and center the data, first, pass the predictors to the `make_column_transformer` function to make the preprocessor.

*Assign your answer to an object called `fruit_preprocessor`.*

In [ ]:
# ___ = make_column_transformer(
#     (___, [___, ___]),
#     verbose_feature_names_out=False
# )

# your code here
raise NotImplementedError
fruit_preprocessor

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_preprocessor is None)).encode("utf-8")+b"bb155").hexdigest() == "b5618d88f851c77f56385ac86f3ffa7b1d3db22b", "type of fruit_preprocessor is None is not bool. fruit_preprocessor is None should be a bool"
assert sha1(str(fruit_preprocessor is None).encode("utf-8")+b"bb155").hexdigest() == "2d85558d02c39bc5e9cc39afac20c33fc75476b4", "boolean value of fruit_preprocessor is None is not correct"

assert sha1(str(type(type(fruit_preprocessor))).encode("utf-8")+b"bb156").hexdigest() == "655cc889db165fb347fc034a624ae106e195f0eb", "type of type(fruit_preprocessor) is not correct"
assert sha1(str(type(fruit_preprocessor)).encode("utf-8")+b"bb156").hexdigest() == "3c63f8d8c24b2bf40155e89f06ab536c295241f1", "value of type(fruit_preprocessor) is not correct"

assert sha1(str(type(fruit_preprocessor.transformers[0][0])).encode("utf-8")+b"bb157").hexdigest() == "fa28239c46b705f0ad589213e9ce6b9160825967", "type of fruit_preprocessor.transformers[0][0] is not str. fruit_preprocessor.transformers[0][0] should be an str"
assert sha1(str(len(fruit_preprocessor.transformers[0][0])).encode("utf-8")+b"bb157").hexdigest() == "25c2eeb1bc500e851b00c107cbd1620e719e74ad", "length of fruit_preprocessor.transformers[0][0] is not correct"
assert sha1(str(fruit_preprocessor.transformers[0][0].lower()).encode("utf-8")+b"bb157").hexdigest() == "1127f69126ad80bc3f387566bc1491535c00948b", "value of fruit_preprocessor.transformers[0][0] is not correct"
assert sha1(str(fruit_preprocessor.transformers[0][0]).encode("utf-8")+b"bb157").hexdigest() == "1127f69126ad80bc3f387566bc1491535c00948b", "correct string value of fruit_preprocessor.transformers[0][0] but incorrect case of letters"

assert sha1(str(type(fruit_preprocessor.transformers[0][2])).encode("utf-8")+b"bb158").hexdigest() == "410934d80b19b6ffc3d8f6fbcde47f427e434f06", "type of fruit_preprocessor.transformers[0][2] is not list. fruit_preprocessor.transformers[0][2] should be a list"
assert sha1(str(len(fruit_preprocessor.transformers[0][2])).encode("utf-8")+b"bb158").hexdigest() == "bacd3266b93071fa6d386dce956d6cbba302e30d", "length of fruit_preprocessor.transformers[0][2] is not correct"
assert sha1(str(sorted(map(str, fruit_preprocessor.transformers[0][2]))).encode("utf-8")+b"bb158").hexdigest() == "d9d40392db757bc0aa082ac21f932b6f53b889bc", "values of fruit_preprocessor.transformers[0][2] are not correct"
assert sha1(str(fruit_preprocessor.transformers[0][2]).encode("utf-8")+b"bb158").hexdigest() == "d1099b29c019486eb564b3693c857f3fe362c532", "order of elements of fruit_preprocessor.transformers[0][2] is not correct"

print('Success!')

**Question 2.2**
<br> {points: 1}

So far, we have split the training and testing datasets as well as preprocessed the data. Now, let's create our K-nearest neighbour classifier with only the training set using the `scikit-learn` package. First, create the classifier by specifying that we want $K = 3$ neighbors. *Assign your answer to an object called `knn_spec`*. 

Next, separate the predictor columns from the target column. Name the predictor variable `X` and the target `y`. 

Train the classifier with the training data set using the `make_pipeline` and `fit` function. The `make_pipeline` function allows you to bundle together your pre-processing, modeling, and post-processing requests. Scaffolding is provided below for you.

*Assign your answer to an object called `fruit_fit`*.

In [ ]:
# ___ = KNeighborsClassifier(n_neighbors=___)

# ___ = ___[["mass", "color_score"]]
# ___ = fruit_train[___]

# ___ = make_pipeline(___, ___).fit(___, ___)

# your code here
raise NotImplementedError
fruit_fit

In [ ]:
from hashlib import sha1
assert sha1(str(type(knn_spec is None)).encode("utf-8")+b"912").hexdigest() == "6a3578d8ffafccfb075e068b1a0e914171c7478e", "type of knn_spec is None is not bool. knn_spec is None should be a bool"
assert sha1(str(knn_spec is None).encode("utf-8")+b"912").hexdigest() == "d986a9bc36e72c3b96e1e2031a5626143fe2da95", "boolean value of knn_spec is None is not correct"

assert sha1(str(type(type(knn_spec))).encode("utf-8")+b"913").hexdigest() == "543b6fdfba1060380533129f4f74328b29b3a0e2", "type of type(knn_spec) is not correct"
assert sha1(str(type(knn_spec)).encode("utf-8")+b"913").hexdigest() == "287ddb7120a8dfda94fb7d2428c8d0c5c375d7b7", "value of type(knn_spec) is not correct"

assert sha1(str(type(knn_spec.effective_metric_)).encode("utf-8")+b"914").hexdigest() == "885b78be77fefd02ca01a4a938b7d9b0b477b051", "type of knn_spec.effective_metric_ is not str. knn_spec.effective_metric_ should be an str"
assert sha1(str(len(knn_spec.effective_metric_)).encode("utf-8")+b"914").hexdigest() == "664a69812634d5c02e2642f29280bef2f7e85f6a", "length of knn_spec.effective_metric_ is not correct"
assert sha1(str(knn_spec.effective_metric_.lower()).encode("utf-8")+b"914").hexdigest() == "c919c40f8cafd40c70f74bdc78c1879559b1deac", "value of knn_spec.effective_metric_ is not correct"
assert sha1(str(knn_spec.effective_metric_).encode("utf-8")+b"914").hexdigest() == "c919c40f8cafd40c70f74bdc78c1879559b1deac", "correct string value of knn_spec.effective_metric_ but incorrect case of letters"

assert sha1(str(type(knn_spec.n_neighbors)).encode("utf-8")+b"915").hexdigest() == "8f6418c8bb2bc0f4663d8b7d6245c4da0e078cbc", "type of knn_spec.n_neighbors is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(knn_spec.n_neighbors).encode("utf-8")+b"915").hexdigest() == "897af3b3a0bf7d1326ec4b728332311a88cec6fa", "value of knn_spec.n_neighbors is not correct"

assert sha1(str(type(sum(X.mass))).encode("utf-8")+b"916").hexdigest() == "96d6af276d9b83fbccada950f012a231a0baaf82", "type of sum(X.mass) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(X.mass)).encode("utf-8")+b"916").hexdigest() == "3813abc610be7d7576e984423bbb40d1e5636c67", "value of sum(X.mass) is not correct"

assert sha1(str(type(sum(X.color_score))).encode("utf-8")+b"917").hexdigest() == "316aa86bbf4789faf93841e135654cc7835dbb61", "type of sum(X.color_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(X.color_score), 2)).encode("utf-8")+b"917").hexdigest() == "fa68f9621eeb4da017b3e938aae0f1d940e36831", "value of sum(X.color_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(y.name)).encode("utf-8")+b"918").hexdigest() == "770c48d59417490a2396e13631c57442b27f9873", "type of y.name is not str. y.name should be an str"
assert sha1(str(len(y.name)).encode("utf-8")+b"918").hexdigest() == "a625d9d63713777c0db4b6a3d1874b86b3d44f67", "length of y.name is not correct"
assert sha1(str(y.name.lower()).encode("utf-8")+b"918").hexdigest() == "d289f00492a6c97b771e7a217ffec2774ab83995", "value of y.name is not correct"
assert sha1(str(y.name).encode("utf-8")+b"918").hexdigest() == "d289f00492a6c97b771e7a217ffec2774ab83995", "correct string value of y.name but incorrect case of letters"

assert sha1(str(type(fruit_fit is None)).encode("utf-8")+b"919").hexdigest() == "ce7bca0fb75e89a7a4ea39263ca6e743f25da53b", "type of fruit_fit is None is not bool. fruit_fit is None should be a bool"
assert sha1(str(fruit_fit is None).encode("utf-8")+b"919").hexdigest() == "b132b7e8cb5ebf4597d526873326d14c4a666e63", "boolean value of fruit_fit is None is not correct"

assert sha1(str(type(type(fruit_fit))).encode("utf-8")+b"91a").hexdigest() == "4b087c8ecdf10a8ec26c168cd9d159ad7b69b75e", "type of type(fruit_fit) is not correct"
assert sha1(str(type(fruit_fit)).encode("utf-8")+b"91a").hexdigest() == "6aed054abd385ed7fa4a8645a06090cec6a1449a", "value of type(fruit_fit) is not correct"

assert sha1(str(type(len(fruit_fit.named_steps))).encode("utf-8")+b"91b").hexdigest() == "237944d67118ff8cd748214bfd37168c26ea997c", "type of len(fruit_fit.named_steps) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(len(fruit_fit.named_steps)).encode("utf-8")+b"91b").hexdigest() == "94079cb0feb9c6c823d64cca3b0e9327aa209860", "value of len(fruit_fit.named_steps) is not correct"

assert sha1(str(type(fruit_fit.named_steps.keys())).encode("utf-8")+b"91c").hexdigest() == "0fa22e41c36c6ca74670299b1cd9208b4f50b201", "type of fruit_fit.named_steps.keys() is not correct"
assert sha1(str(fruit_fit.named_steps.keys()).encode("utf-8")+b"91c").hexdigest() == "d9af1087666a5e17a3c5cb819d067533943ee967", "value of fruit_fit.named_steps.keys() is not correct"

print('Success!')

**Question 2.3**
<br> {points: 1}

Now that we have created our K-nearest neighbor classifier object, let's predict the class labels for our test set.

We want to make sure to `assign` the predicted class labels to a new column in the dataframe, called `predicted`. To create the predicted class labels pass your fitted model pipeline and the **test dataset** to the `predict` function. 

*Assign your answer to an object called `fruit_test_predictions`.*

In [ ]:
# ___ = fruit_test.___(
#     predicted=___.predict(___[[___, ___]])
# )

# your code here
raise NotImplementedError
fruit_test_predictions

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_test_predictions is None)).encode("utf-8")+b"4553f").hexdigest() == "1c8e9d20d9cf368c8f659e6270921645f58cb550", "type of fruit_test_predictions is None is not bool. fruit_test_predictions is None should be a bool"
assert sha1(str(fruit_test_predictions is None).encode("utf-8")+b"4553f").hexdigest() == "b827a7368a90ccda88fa46a4a784ee684a0a1bb8", "boolean value of fruit_test_predictions is None is not correct"

assert sha1(str(type(fruit_test_predictions)).encode("utf-8")+b"45540").hexdigest() == "9851399a3b8039e51342d71f2fc2a6f698fb3e89", "type of type(fruit_test_predictions) is not correct"

assert sha1(str(type(fruit_test_predictions.shape)).encode("utf-8")+b"45541").hexdigest() == "60405e28d966c6b69b0e2b57f53650e8efbcf01d", "type of fruit_test_predictions.shape is not tuple. fruit_test_predictions.shape should be a tuple"
assert sha1(str(len(fruit_test_predictions.shape)).encode("utf-8")+b"45541").hexdigest() == "d7035e7fcbf882bfd3794cb1981f17fde8712087", "length of fruit_test_predictions.shape is not correct"
assert sha1(str(sorted(map(str, fruit_test_predictions.shape))).encode("utf-8")+b"45541").hexdigest() == "2e339df7b65312d1eab89c428b436f9818f51fdd", "values of fruit_test_predictions.shape are not correct"
assert sha1(str(fruit_test_predictions.shape).encode("utf-8")+b"45541").hexdigest() == "8606ba2cd9314fed9a5ac370e704936677519a81", "order of elements of fruit_test_predictions.shape is not correct"

assert sha1(str(type("predicted" in fruit_test_predictions.columns)).encode("utf-8")+b"45542").hexdigest() == "8f14e690efcaf1fad9240c18e2dfa901ba592629", "type of \"predicted\" in fruit_test_predictions.columns is not bool. \"predicted\" in fruit_test_predictions.columns should be a bool"
assert sha1(str("predicted" in fruit_test_predictions.columns).encode("utf-8")+b"45542").hexdigest() == "9399fd333362d5688302d4f1a9dbc6df86de1134", "boolean value of \"predicted\" in fruit_test_predictions.columns is not correct"

print('Success!')

**Question 2.4**
<br> {points: 1}

Great! We have now computed some predictions for our test datasets! From glancing at the dataframe above, it looks like most of them are correct, but wouldn't it be interesting if we could find out the exact accuracy of our classifier? 

Thankfully, the `score` function from the `scikit-learn` package can help us. To get the statistics about the quality of our model, you need to call the `score` function on the `fruit_fit` model. Name the predictors `X_test` and the target `y_test`. We should pass the `X_test` and `y_test` into the `score` function.

*Assign your answer to an object called `fruit_prediction_accuracy`.*

In [ ]:
# ___ = ___[[___, ___]]
# ___ = ___["fruit_name"]

# ___ = fruit_fit.score(___, ___)

# your code here
raise NotImplementedError
fruit_prediction_accuracy

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_prediction_accuracy is None)).encode("utf-8")+b"3096c").hexdigest() == "ce5c3ae0b8cabef3be16f3b22f77971c5090bfc1", "type of fruit_prediction_accuracy is None is not bool. fruit_prediction_accuracy is None should be a bool"
assert sha1(str(fruit_prediction_accuracy is None).encode("utf-8")+b"3096c").hexdigest() == "f63cbfdb12c860df3e54ed11bc9bdc1600b07ab6", "boolean value of fruit_prediction_accuracy is None is not correct"

assert sha1(str(type(fruit_prediction_accuracy)).encode("utf-8")+b"3096d").hexdigest() == "8cb1e1ffad638e2bcb21ef4c1b70b550638f4eca", "type of fruit_prediction_accuracy is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(fruit_prediction_accuracy, 2)).encode("utf-8")+b"3096d").hexdigest() == "7f1e8e4a62b7d4b308b276fc484e2a490d9865b5", "value of fruit_prediction_accuracy is not correct (rounded to 2 decimal places)"

print('Success!')

**Question 2.5**
<br> {points: 1}

Now, let's look at the *confusion matrix* for the classifier. This will show us a table comparing the predicted labels with the true labels. 

A confusion matrix is essentially a classification matrix. The columns of the confusion matrix represent the actual class and the rows represent the predicted class (or vice versa). Shown below is an example of a confusion matrix.

|                    | Predicted Positive | Predicted Negative |
|--------------------|:------------------:|:------------------:|
| **Truly Positive** | True Positive      |     False Negative |
| **Truly Negative** | False Positive     |      True Negative |


- A **true positive** is an outcome where the model correctly predicts the positive class.
- A **true negative** is an outcome where the model correctly predicts the negative class.
- A **false positive** is an outcome where the model incorrectly predicts the positive class.
- A **false negative** is an outcome where the model incorrectly predicts the negative class.

<br>

We can create a confusion matrix by using the `crosstab` function from `pandas`. In the dataframe created by `crosstab`, the true labels will be to the left, and the predicted labels will be on top (as in the matrix above). In contrast to the confusion matrix above where there are only two possible outcomes (positive/negative), we have four possible outcomes (the four fruit names). Therefore, our dataframe will be bigger than the matrix above and contain 16 possible outcomes instead of 4.

*Assign your answer to an object called `fruit_mat`*.

In [ ]:
# ___ = pd.___(
#     fruit_test_predictions[___],  # True labels
#     fruit_test_predictions[___],  # Predicted labels
# )

# your code here
raise NotImplementedError
fruit_mat

With many observations, it can be difficult to interpret the confusion matrix when it is presented as a table like above. In these cases, we could instead use the `ConfusionMatrixDisplay` function of the `scikit-learn` package to visualize the confusion matrix as a heatmap. Please run the cell below to see the fruit confusion matrix as a heatmap.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(
    fruit_fit,  # We are directly passing the pipeline and let sklearn do the predictions for us
    X_test,
    y_test
)

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_mat is None)).encode("utf-8")+b"ce932").hexdigest() == "fbb889ef0c4dd983ec8d6a555a0e6de6714ad416", "type of fruit_mat is None is not bool. fruit_mat is None should be a bool"
assert sha1(str(fruit_mat is None).encode("utf-8")+b"ce932").hexdigest() == "435ce0003da7f12939dab1bdc33d259dc80eb1aa", "boolean value of fruit_mat is None is not correct"

assert sha1(str(type(fruit_mat)).encode("utf-8")+b"ce933").hexdigest() == "d8f2899b881cd3197058edc6b909386b878196f4", "type of type(fruit_mat) is not correct"

assert sha1(str(type(fruit_mat.to_numpy().sum())).encode("utf-8")+b"ce934").hexdigest() == "06972e32f652c9422376ed3d4fd2dff423167e86", "type of fruit_mat.to_numpy().sum() is not correct"
assert sha1(str(fruit_mat.to_numpy().sum()).encode("utf-8")+b"ce934").hexdigest() == "c398ce427aea303fff7f87e2e7a07cdd3a239ed2", "value of fruit_mat.to_numpy().sum() is not correct"

print('Success!')

**Question 2.6** Multiple Choice:
<br> {points: 1}

Reading `fruit_mat`, how many observations were labelled correctly?

A. 7

B. 8

C. 9

D. 14

*Assign your answer to an object called `answer2_6`. Make sure your answer is an uppercase letter and is surrounded by quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError
answer2_6

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer2_6)).encode("utf-8")+b"148a1").hexdigest() == "346a5a1312f0288cf3999a50489f4c86859bec50", "type of answer2_6 is not str. answer2_6 should be an str"
assert sha1(str(len(answer2_6)).encode("utf-8")+b"148a1").hexdigest() == "8618984b0957ac448e6d0aff7a90e9a7abf694f7", "length of answer2_6 is not correct"
assert sha1(str(answer2_6.lower()).encode("utf-8")+b"148a1").hexdigest() == "75a429662cedc9d1ce34d3b9024d852f669aa60f", "value of answer2_6 is not correct"
assert sha1(str(answer2_6).encode("utf-8")+b"148a1").hexdigest() == "c98bde0f6127520e68eea850c5668a684beecc9f", "correct string value of answer2_6 but incorrect case of letters"

print('Success!')

### 3. Cross-validation

**Question 3.1**
<br> {points: 1}

The vast majority of predictive models in statistics and machine learning have parameters that you have to pick. For the past few exercises, we have had to pick the number of neighbours for the class vote, which we have done arbitraily. But, is it possible to make this selection, *i.e., tune the model, in a principled way?* Ideally, we want to pick the number of neighborurs to maximize the performance of our classifier on data *it hasn’t seen yet*.

An important aspect of the tuning process is that we can, if we want to, split our training data again, train and evaluate a classifier for each split, and then choose the parameter based on all of the different results. If we just split our training data once, our best parameter choice will depend strongly on the randomness from how this single split was made. Using multiple different splits, we’ll get a more robust estimate of accuracy, which will lead to a more suitable choice of the number of neighbours $K$ to perform well on unseen data.

The idea of training and evaluating models on multiple training data splits times is called "cross-validation". In cross-validation, we split our overall training data into $C$ evenly-sized chunks, and then iteratively use 1 chunk as the **validation set** and combine the remaining $C−1$ chunks as the **training set.** The validation set is used in a similar was as the test set, **except** that the test set is only used once at the end to report model performance whether we use model performance on the validation set to select the model during cross-validation. 

---

We can perform a cross-validation in Python using the `cross_validate` function from the `scikit-learn` package. To use this function, you have to identify the model, the training set, and specify the `cv` parameter (the number of folds $C$, defaults to 5). We should set `return_train_score` to be `True` to return the training score as well.

Before we use the `cross_validate` function, we need to perform the pipeline analysis again. You can reuse the `X` and `y` variables you constructed from the training data earlier, as well as the `fruit_preprocessor`and `knn_spec` variables. However, you will need to create a new pipeline since the one we made earlier is already fitted on all the data and here we want to fit it on different splits of the data during cross-validation. Since the cross_validate function outputs a dictionary, we use `pd.DataFrame` to convert it to a dataframe for convenience, as in the textbook.

*Assign your answer to an object called `fruit_vfold_score`*.

In [ ]:
np.random.seed(2020)  # DO NOT REMOVE

# ___ = ___(fruit_preprocessor, knn_spec)
# ___ = pd.___(
#     cross_validate(
#         estimator=___,
#         cv=5,
#         X=___,
#         y=___,
#         return_train_score=True,
#     )
# )

# your code here
raise NotImplementedError
fruit_vfold_score

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_vfold_score is None)).encode("utf-8")+b"ea6ad").hexdigest() == "ff9b31da721e12c8d1ca9670a515b4d404731a39", "type of fruit_vfold_score is None is not bool. fruit_vfold_score is None should be a bool"
assert sha1(str(fruit_vfold_score is None).encode("utf-8")+b"ea6ad").hexdigest() == "76354876a712d0233045b4b7643353b7dd64c151", "boolean value of fruit_vfold_score is None is not correct"

assert sha1(str(type(fruit_vfold_score)).encode("utf-8")+b"ea6ae").hexdigest() == "f7ca54425c83ae803141ac213847fc8b21e929af", "type of type(fruit_vfold_score) is not correct"

assert sha1(str(type(fruit_vfold_score.shape)).encode("utf-8")+b"ea6af").hexdigest() == "e1df773b88ec10fa845799c88959e69357ee4221", "type of fruit_vfold_score.shape is not tuple. fruit_vfold_score.shape should be a tuple"
assert sha1(str(len(fruit_vfold_score.shape)).encode("utf-8")+b"ea6af").hexdigest() == "cecb65427ddbbec2bb9f7da8b20601a8d99913fd", "length of fruit_vfold_score.shape is not correct"
assert sha1(str(sorted(map(str, fruit_vfold_score.shape))).encode("utf-8")+b"ea6af").hexdigest() == "728ca34fae61511d0a3a1fa51704524d0d84739a", "values of fruit_vfold_score.shape are not correct"
assert sha1(str(fruit_vfold_score.shape).encode("utf-8")+b"ea6af").hexdigest() == "b71ab81f5076019a389db0edf7dae23d0e91233e", "order of elements of fruit_vfold_score.shape is not correct"

assert sha1(str(type(fruit_pipe is None)).encode("utf-8")+b"ea6b0").hexdigest() == "f0cdf19810b1b09a3cc5e0cd95a9a52003b5aa6e", "type of fruit_pipe is None is not bool. fruit_pipe is None should be a bool"
assert sha1(str(fruit_pipe is None).encode("utf-8")+b"ea6b0").hexdigest() == "31b42fb653bfaf7efb6719ae628d31ce59a97cdc", "boolean value of fruit_pipe is None is not correct"

assert sha1(str(type(type(fruit_pipe))).encode("utf-8")+b"ea6b1").hexdigest() == "12c2773484a78a5409505d95a43c3e7dba2b2cbf", "type of type(fruit_pipe) is not correct"
assert sha1(str(type(fruit_pipe)).encode("utf-8")+b"ea6b1").hexdigest() == "3546e6f2e6875cf844b967515d53c167d5c4931f", "value of type(fruit_pipe) is not correct"

assert sha1(str(type(len(fruit_pipe.named_steps))).encode("utf-8")+b"ea6b2").hexdigest() == "282bb12bfe45e99532bf6af3b5594ecac3217fb9", "type of len(fruit_pipe.named_steps) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(len(fruit_pipe.named_steps)).encode("utf-8")+b"ea6b2").hexdigest() == "5c5f2949d17eb3d80e0cd467a2fef62dc5f8dd7a", "value of len(fruit_pipe.named_steps) is not correct"

assert sha1(str(type(fruit_pipe.named_steps.keys())).encode("utf-8")+b"ea6b3").hexdigest() == "4c2523cdb5356030ff0b00362db1a5053df1fa61", "type of fruit_pipe.named_steps.keys() is not correct"
assert sha1(str(fruit_pipe.named_steps.keys()).encode("utf-8")+b"ea6b3").hexdigest() == "3f84919af8539cf5590c5f15ee4437bd1b4c2eff", "value of fruit_pipe.named_steps.keys() is not correct"

print('Success!')

**Question 3.2**
<br> {points: 1}

Now that we have ran a cross-validation on each train/validation split, one has to ask, how accurate was the classifier's validation across the folds? We can aggregate the *mean* and *standard error* of these scores from each folds. The standard error is essentially a measure of how uncertain we are in the mean value. Use the `agg` dataframe method to compute both the mean and the standard error; make sure the first row of the dataframe contains the mean values and the second contains the standard error values.

*Assign your answer to an object called `fruit_metrics`.*

In [ ]:
# ___ = fruit_vfold_score.___([___, ___])


# your code here
raise NotImplementedError
fruit_metrics

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_metrics.shape)).encode("utf-8")+b"3e87a").hexdigest() == "1034273ae261d186e8a04af350d63ec450f398ce", "type of fruit_metrics.shape is not tuple. fruit_metrics.shape should be a tuple"
assert sha1(str(len(fruit_metrics.shape)).encode("utf-8")+b"3e87a").hexdigest() == "4f793c542cbaddc831541a2361afc2f348215df7", "length of fruit_metrics.shape is not correct"
assert sha1(str(sorted(map(str, fruit_metrics.shape))).encode("utf-8")+b"3e87a").hexdigest() == "6abfca6c1c33716d6ffaa35c0c54589a0e01cc49", "values of fruit_metrics.shape are not correct"
assert sha1(str(fruit_metrics.shape).encode("utf-8")+b"3e87a").hexdigest() == "efc2eb50b133952f621823ffed80a7bac85e4e3f", "order of elements of fruit_metrics.shape is not correct"

assert sha1(str(type(fruit_metrics)).encode("utf-8")+b"3e87b").hexdigest() == "76088d59f6d5bd59e7a9b2ffdf423d86ea80de20", "type of fruit_metrics is not correct"
assert sha1(str(fruit_metrics).encode("utf-8")+b"3e87b").hexdigest() == "657e85bfc3eb001a5a497439957f1b28745227d0", "value of fruit_metrics is not correct"

assert sha1(str(type(fruit_metrics.test_score)).encode("utf-8")+b"3e87c").hexdigest() == "00f65b8828925d9f06e60a676793547ad1bb166b", "type of fruit_metrics.test_score is not correct"
assert sha1(str(fruit_metrics.test_score).encode("utf-8")+b"3e87c").hexdigest() == "5664072d8b7565b1228da370c2a90f007c9a74ed", "value of fruit_metrics.test_score is not correct"

print('Success!')

## 4. Parameter value selection

Using a 5-fold cross-validation, we have established a prediction accuracy for our classifier. If we were to improve our classifier, we would like to try different number of neighbours, $K$. Then we could use cross-validation to calculate an accuracy for each value of $K$ in a reasonable range, and pick the value of $K$ that gives us the best accuracy on the validation data. 

The great thing about the `scikit-learn` package is that it provides functions to conveniently tune parameters such as $K$ by training and evaluating models (via crossvalidation) for a range of specified values of $K$. The function we will use here is called "exhaustive grid search" ([sklearn.model_selection.GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).

**Question 4.0**
<br> {points: 1}

Create a new K-nearest neighbor model specification but instead of specifying a particular value for the `n_neighbors` argument, try exploring a range of values with `GridSearchCV`. Before we use `GridSearchCV`, we should define the grid of values that we want to explore, and redefine the pipeline without specifying a particular value of $K$. To save us some time, instruct the grid search to use 4-fold cross-validation, rather than the default 5-fold.

*Assign your answer to an object called `knn_tune_grid`.* 

In [ ]:
### Run this cell
param_grid = {
    "kneighborsclassifier__n_neighbors": range(2, 15, 1),
}
fruit_tune_pipe = make_pipeline(fruit_preprocessor, KNeighborsClassifier())

In [ ]:
# ___ = GridSearchCV(
#     ___, ___, ___=__,
# )


# your code here
raise NotImplementedError
knn_tune_grid

In [ ]:
from hashlib import sha1
assert sha1(str(type(knn_tune_grid is None)).encode("utf-8")+b"49a8f").hexdigest() == "d6e3cead40716424a6b785f12cdfa1019457d0fa", "type of knn_tune_grid is None is not bool. knn_tune_grid is None should be a bool"
assert sha1(str(knn_tune_grid is None).encode("utf-8")+b"49a8f").hexdigest() == "992737945795abcf07a22d9de5c761eccc0bdcb1", "boolean value of knn_tune_grid is None is not correct"

assert sha1(str(type(type(knn_tune_grid))).encode("utf-8")+b"49a90").hexdigest() == "6cea27e07e996650936e5c8593e635150ffb0a44", "type of type(knn_tune_grid) is not correct"
assert sha1(str(type(knn_tune_grid)).encode("utf-8")+b"49a90").hexdigest() == "f3770e7f79904cf07abea4bd0d9f52bafd32528c", "value of type(knn_tune_grid) is not correct"

assert sha1(str(type(knn_tune_grid.param_grid.keys())).encode("utf-8")+b"49a91").hexdigest() == "0e5f74303563f3ce4fca3706d9634b8ac10c6a37", "type of knn_tune_grid.param_grid.keys() is not correct"
assert sha1(str(knn_tune_grid.param_grid.keys()).encode("utf-8")+b"49a91").hexdigest() == "9d7a16f3c9efa544ad60651e5920d731490d885a", "value of knn_tune_grid.param_grid.keys() is not correct"

assert sha1(str(type(knn_tune_grid.estimator.named_steps.keys())).encode("utf-8")+b"49a92").hexdigest() == "ee19a431527fbca60576cd47c26cfa1822c6bfcc", "type of knn_tune_grid.estimator.named_steps.keys() is not correct"
assert sha1(str(knn_tune_grid.estimator.named_steps.keys()).encode("utf-8")+b"49a92").hexdigest() == "dfea5bbce1ff10ef41d542caa7c08c0e0864ecc7", "value of knn_tune_grid.estimator.named_steps.keys() is not correct"

assert sha1(str(type(knn_tune_grid.cv)).encode("utf-8")+b"49a93").hexdigest() == "144965142d54e68c18ad9e9d768ad0314da973c2", "type of knn_tune_grid.cv is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(knn_tune_grid.cv).encode("utf-8")+b"49a93").hexdigest() == "d6b253aca5ea4ef5545014c34821043dcae02802", "value of knn_tune_grid.cv is not correct"

print('Success!')

**Question 4.1**
<br>{points: 1}

Now, let's fit the grid search object to the data, using the `X` and `y` variables we created earlier.

*Assign your tuned model to a variable called `knn_model_grid`.*

Next, from `knn_model_grid`, find out the `cv_results_` and save it in a dataframe. 

*Assign your answer to a variable called `accuracies_grid`.*

In [ ]:
# ___ = ___.fit(___, ___)

# ___ = pd.DataFrame(___.cv_results_)

# your code here
raise NotImplementedError
accuracies_grid

In [ ]:
from hashlib import sha1
assert sha1(str(type(type(knn_model_grid))).encode("utf-8")+b"b70d9").hexdigest() == "c9e85210eeddf35d1608039372fbd66d03bc7631", "type of type(knn_model_grid) is not correct"
assert sha1(str(type(knn_model_grid)).encode("utf-8")+b"b70d9").hexdigest() == "1da9107711be6d87a1633cd5208cd3186591055b", "value of type(knn_model_grid) is not correct"

assert sha1(str(type(accuracies_grid is None)).encode("utf-8")+b"b70da").hexdigest() == "fbc6dddc2b05c39483ae91a835b5297edbdb9812", "type of accuracies_grid is None is not bool. accuracies_grid is None should be a bool"
assert sha1(str(accuracies_grid is None).encode("utf-8")+b"b70da").hexdigest() == "c7ed2ca177a9a7f72663038ec3c9af33ecb008b6", "boolean value of accuracies_grid is None is not correct"

assert sha1(str(type(accuracies_grid)).encode("utf-8")+b"b70db").hexdigest() == "3daa084b3639c9363a36c1cae0b5af2669d0a5d3", "type of type(accuracies_grid) is not correct"

assert sha1(str(type(accuracies_grid.shape)).encode("utf-8")+b"b70dc").hexdigest() == "3fff8eec3dced6045bf90eebc84f13bd238ed955", "type of accuracies_grid.shape is not tuple. accuracies_grid.shape should be a tuple"
assert sha1(str(len(accuracies_grid.shape)).encode("utf-8")+b"b70dc").hexdigest() == "70419412b8182c8997d65294f9733599ebd8fe13", "length of accuracies_grid.shape is not correct"
assert sha1(str(sorted(map(str, accuracies_grid.shape))).encode("utf-8")+b"b70dc").hexdigest() == "379ac44ff1bedf6754b59921b03e048657098ca4", "values of accuracies_grid.shape are not correct"
assert sha1(str(accuracies_grid.shape).encode("utf-8")+b"b70dc").hexdigest() == "c5bc31f7edfd904b4f7c1e70b53ffe60be5678f7", "order of elements of accuracies_grid.shape is not correct"

assert sha1(str(type(sum(accuracies_grid.mean_test_score))).encode("utf-8")+b"b70dd").hexdigest() == "8a906d8d5e91bdfda85045bc5008b426a67c64fc", "type of sum(accuracies_grid.mean_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_grid.mean_test_score), 2)).encode("utf-8")+b"b70dd").hexdigest() == "5707983c667d6ac0f92517335b53e577df84a3ce", "value of sum(accuracies_grid.mean_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_grid.std_test_score))).encode("utf-8")+b"b70de").hexdigest() == "8863f5d560871669596a1931b5f20dde7d26a8f6", "type of sum(accuracies_grid.std_test_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(accuracies_grid.std_test_score), 2)).encode("utf-8")+b"b70de").hexdigest() == "53c7c788a42f6bd2ab358e8a16b7a2f3b342d687", "value of sum(accuracies_grid.std_test_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(accuracies_grid.param_kneighborsclassifier__n_neighbors))).encode("utf-8")+b"b70df").hexdigest() == "b6ec2b53dd224062feaa03dce68f6cd6db4f7200", "type of sum(accuracies_grid.param_kneighborsclassifier__n_neighbors) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(accuracies_grid.param_kneighborsclassifier__n_neighbors)).encode("utf-8")+b"b70df").hexdigest() == "ac1537ceced74954a371a29decac2f0d8f0ba206", "value of sum(accuracies_grid.param_kneighborsclassifier__n_neighbors) is not correct"

print('Success!')

**Question 4.2**
<br>{points: 1} 


Visually inspecting the grid search results can help us find the best value for the number of neighbors parameter.

Create a line plot using the `accuracies_grid` dataframe with `param_kneighborsclassifier__n_neighbors` on the x-axis and the `mean_test_score` on the y-axis. Use `point=True` to include a point for each value of $K$. Make it an effective visualization.

*Assign your answer to a variable called `accuracy_versus_k_grid`.*

In [ ]:
# ___ = alt.Chart(___).mark_line(___).encode(
#     x=alt.X(___)
#         .title(___)
#         .scale(zero=False),
#     y=alt.Y(___)
#         .title(___)
#         .scale(zero=False)
# )


# your code here
raise NotImplementedError
accuracy_versus_k_grid

In [ ]:
from hashlib import sha1
assert sha1(str(type(accuracy_versus_k_grid is None)).encode("utf-8")+b"ce2f4").hexdigest() == "f473970694a3f1f1d606b5bc9f1d3bc67aba2726", "type of accuracy_versus_k_grid is None is not bool. accuracy_versus_k_grid is None should be a bool"
assert sha1(str(accuracy_versus_k_grid is None).encode("utf-8")+b"ce2f4").hexdigest() == "60e500358908afb9e46600fe2d775e535172a433", "boolean value of accuracy_versus_k_grid is None is not correct"

assert sha1(str(type(accuracy_versus_k_grid.encoding.x['shorthand'])).encode("utf-8")+b"ce2f5").hexdigest() == "49db110ef57e74cdc4cd6ed16e5922d8c982445b", "type of accuracy_versus_k_grid.encoding.x['shorthand'] is not str. accuracy_versus_k_grid.encoding.x['shorthand'] should be an str"
assert sha1(str(len(accuracy_versus_k_grid.encoding.x['shorthand'])).encode("utf-8")+b"ce2f5").hexdigest() == "585d44923f1cc938a56d7b9eeec5b8627192b63c", "length of accuracy_versus_k_grid.encoding.x['shorthand'] is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.x['shorthand'].lower()).encode("utf-8")+b"ce2f5").hexdigest() == "8c148267aac1bb120cf490ee2877f90545d90638", "value of accuracy_versus_k_grid.encoding.x['shorthand'] is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.x['shorthand']).encode("utf-8")+b"ce2f5").hexdigest() == "8c148267aac1bb120cf490ee2877f90545d90638", "correct string value of accuracy_versus_k_grid.encoding.x['shorthand'] but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_grid.encoding.y['shorthand'])).encode("utf-8")+b"ce2f6").hexdigest() == "f6e3c99ddebb4c2953ae6ba97e700f65e0bee296", "type of accuracy_versus_k_grid.encoding.y['shorthand'] is not str. accuracy_versus_k_grid.encoding.y['shorthand'] should be an str"
assert sha1(str(len(accuracy_versus_k_grid.encoding.y['shorthand'])).encode("utf-8")+b"ce2f6").hexdigest() == "ea18f375087d74df5ec79347f2eb99759e0c850e", "length of accuracy_versus_k_grid.encoding.y['shorthand'] is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.y['shorthand'].lower()).encode("utf-8")+b"ce2f6").hexdigest() == "e390c7e0756b755c0cd703930f08d3f2cb06026b", "value of accuracy_versus_k_grid.encoding.y['shorthand'] is not correct"
assert sha1(str(accuracy_versus_k_grid.encoding.y['shorthand']).encode("utf-8")+b"ce2f6").hexdigest() == "e390c7e0756b755c0cd703930f08d3f2cb06026b", "correct string value of accuracy_versus_k_grid.encoding.y['shorthand'] but incorrect case of letters"

assert sha1(str(type(accuracy_versus_k_grid.mark)).encode("utf-8")+b"ce2f7").hexdigest() == "4a20aaa06c0473ffd1f6373f8b5b77ea65411201", "type of accuracy_versus_k_grid.mark is not correct"
assert sha1(str(accuracy_versus_k_grid.mark).encode("utf-8")+b"ce2f7").hexdigest() == "a03c0f2d87ea20413ace02738affb7bc184ecf9a", "value of accuracy_versus_k_grid.mark is not correct"

assert sha1(str(type(accuracy_versus_k_grid.mark['point'])).encode("utf-8")+b"ce2f8").hexdigest() == "4e269e94d9fd9d43b74bd6ecb53e42b9747c651a", "type of accuracy_versus_k_grid.mark['point'] is not bool. accuracy_versus_k_grid.mark['point'] should be a bool"
assert sha1(str(accuracy_versus_k_grid.mark['point']).encode("utf-8")+b"ce2f8").hexdigest() == "3d796ac5ac10952db5a902f6ccd0fe3d8f9b97cf", "boolean value of accuracy_versus_k_grid.mark['point'] is not correct"

print('Success!')

From the plots above, we can see that $K = 2$ or $3$ provides the highest accuracy. Larger $K$ values result in a reduced accuracy estimate. Remember: the values you see on this plot are **estimates** of the true accuracy of our classifier. Although this is the best information we have access to for what the ideal value of $K$ would be, it is  not a gurantee that the classifier will always be more accurate with this parameter value when it is used in practice! 

Great, now you have completed a full analysis with cross-validation using the `scikit-learn` package! For your information, we can choose any number of folds and typically, the more we use the better our accuracy estimate will be (lower standard error). However, more folds would mean a greater computation time. In practice, $cv$ is chosen to be either 5 or 10. 